In [ ]:
import biospat.biospat as biospat_full
from biospat.stand_alone_functions import (
    merge_touching_groups,
    classify_range_edges,
    assign_polygon_clusters,
    update_polygon_categories,
)

In [ ]:
import geopandas as gpd


new_map = biospat_full.Map()

# Load your range polygons and lakes
new_map.load_historic_data("Acer negundo")  # Replace with your actual species name

# Remove lakes from the range polygons
range_no_lakes = new_map.remove_lakes(new_map.gdfs["acernegu"])

merged_polygons = merge_touching_groups(range_no_lakes, buffer_distance=5000)

island_states_gdf = gpd.read_file("../../island_states.gpkg")

# Find and cluster polygons

clustered_polygons, largest_polygons = assign_polygon_clusters(
    merged_polygons, island_states_gdf
)

# clustered_polygons.plot(column='cluster', figsize=(10, 10))

classified_polygons = classify_range_edges(clustered_polygons, largest_polygons)

updated_polygon = update_polygon_categories(
    largest_polygons, classified_polygons, island_states_gdf
)

updated_polygon.plot(column="category", figsize=(10, 10), legend=True)